# Задание 2. Работа с API

Есть эндпоинты: https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap 
https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false).

Задачи:
1. Напишите скрипт, который выведет монеты из эндпоинта coinmarketcap, которых нет в списке доступных у обменника https://simpleswap.io.
2. Положите этот список в базу.
3. Отсортируйте токены и монеты по суточному обороту по убыванию.
4. Обязательно убедитесь, что список верный.



## Выгрузка данных

In [2]:
import requests
import json
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
endpoint_1 = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap"

In [5]:
result_1 = requests.get(endpoint_1).json()

In [6]:
cc_colums = ["name", "volume24h", "marketCap", "cc_id", "cc_name", "cc_symbol", "cc_dateAdded"]

In [7]:
cclist_data = pd.json_normalize(result_1['data']['cryptoCurrencyList'], 'quotes', ["id", "name", "symbol", "dateAdded"], 'cc_')[cc_colums]
cclist_data = cclist_data[(cclist_data['name'] == 'USD')].reset_index(drop=True)
cclist_data = cclist_data.sort_values(["cc_symbol", "cc_dateAdded", "marketCap"], ascending=False).groupby("cc_symbol", as_index=False).first()

In [8]:
cclist_data = cclist_data.drop_duplicates(subset='cc_symbol')

In [9]:
cclist_data[cclist_data.duplicated(subset='cc_symbol', keep=False)].sort_values('cc_symbol')

,cc_symbol,name,volume24h,marketCap,cc_id,cc_name,cc_dateAdded


In [13]:
cclist_data.shape

(1471, 7)

In [14]:
cclist_data.head()

,cc_symbol,name,volume24h,marketCap,cc_id,cc_name,cc_dateAdded
0,$MICHI,USD,2.331537e+06,6.198767e+06,30943,michi (SOL),2024-04-26T07:32:25.000Z
1,$PURPE,USD,1.308348e+06,1.050949e+07,32171,Purple Pepe,2024-07-10T07:05:51.000Z
2,0G,USD,6.200287e+07,3.274696e+08,38337,0G,2025-09-10T06:05:00.000Z
3,0x0,USD,1.989380e+03,8.459489e+06,23355,0x0.ai,2023-01-31T10:00:24.000Z
4,10SET,USD,1.532136e+05,7.003860e+06,9089,Tenset,2021-04-01T00:00:00.000Z


In [15]:
endpoint_2 = "https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false"

In [16]:
result_2 = requests.get(endpoint_2).json()

In [17]:
ss_data = pd.DataFrame(result_2)

In [18]:
ss_data.columns

Index(['id', 'symbol', 'symbolFront', 'name', 'description', 'network',
       'cmcTicker', 'tickerSeo', 'networkSeo', 'isMainnetSeo', 'mainUrl',
       'addressExplorer', 'confirmationsFrom', 'enabled', 'extraId',
       'hasExtraId', 'howToExchangePageUrl', 'isFiat',
       'pricePredictionPageUrl', 'priority', 'priorityFrom', 'priorityTo',
       'supportsFiatExchange', 'txExplorer', 'validationAddress',
       'validationExtra', 'video', 'warningsFrom', 'warningsTo', 'createdAt',
       'ticker', 'price', 'extendedInfo'],
      dtype='object')

In [19]:
ss_data.head()

,id,symbol,symbolFront,name,description,network,cmcTicker,tickerSeo,networkSeo,isMainnetSeo,mainUrl,addressExplorer,confirmationsFrom,enabled,extraId,hasExtraId,howToExchangePageUrl,isFiat,pricePredictionPageUrl,priority,priorityFrom,priorityTo,supportsFiatExchange,txExplorer,validationAddress,validationExtra,video,warningsFrom,warningsTo,createdAt,ticker,price,extendedInfo
0,48395,usdcassethub,None,USDC (DotAssetHub),None,statemint,usdc,None,None,False,None,https://assethub-polkadot.subscan.io/account/{},NaN,True,None,False,None,False,None,False,0,0,None,https://assethub-polkadot.subscan.io/account/{},"^(1)[0-9a-z-A-Z]{44,50}$",None,None,[],[],2025-10-10T14:05:01.058Z,usdc,0.9998320364631815,None
1,48393,vlrop,None,Velora,None,optimism,vlr,None,None,False,None,https://optimistic.etherscan.io/address/{},NaN,True,None,False,None,False,None,False,0,0,None,https://optimistic.etherscan.io/tx/{},^(0x)[0-9A-Fa-f]{40}$,None,None,[],[],2025-10-06T14:10:09.652Z,vlr,0,None
2,48392,vlrbase,None,Velora,None,base,vlr,None,None,False,None,https://basescan.org/address/{},NaN,True,None,False,None,False,None,False,0,0,None,https://basescan.org/tx/{},^(0x)[0-9A-Fa-f]{40}$,None,None,[],[],2025-10-06T14:10:09.647Z,vlr,0,None
3,48391,usdt0,None,USDT0,None,plasma,usdt0,None,None,False,None,https://plasmascan.to/address/{},NaN,True,None,False,None,False,None,False,0,0,None,https://plasmascan.to/tx/{},^(0x)[0-9A-Fa-f]{40}$,None,None,[],[],2025-10-06T14:10:09.640Z,usdt0,0,None
4,48394,usdebsc,None,Ethena USDe,None,bsc,usde,None,None,False,None,https://bscscan.com/address/{},NaN,True,None,False,None,False,None,False,0,0,None,https://bscscan.com/tx/{},^(0x)[0-9A-Fa-f]{40}$,None,None,[],[],2025-10-06T14:10:09.634Z,usde,0,None


## Сравнение источников

In [20]:
def prepare_data(data, col_name, save_col=[], new_name=None):
    data = data[[col_name] + save_col].copy()
    data[col_name] = data[col_name].str.upper()
    if new_name:
        data.rename({col_name: new_name}, axis=1, inplace=True)
    return data

In [21]:
cc_tokens = prepare_data(cclist_data, "cc_symbol", ['cc_id', 'cc_name', "volume24h"])
ss_tokens = prepare_data(ss_data, "symbol", ["id"], 'ss_symbol')

In [22]:
cc_tokens.shape

(1471, 4)

In [23]:
ss_tokens.shape

(2774, 2)

In [24]:
total_tokens = cc_tokens.merge(ss_tokens, left_on='cc_symbol', right_on='ss_symbol', how="left")

In [37]:
missed_tokens = total_tokens[total_tokens['ss_symbol'].isna()][['cc_id', 'cc_name', 'cc_symbol', 'volume24h']]

In [38]:
missed_tokens.sort_values('volume24h', ascending=False, inplace=True)

In [39]:
missed_tokens

,cc_id,cc_name,cc_symbol,volume24h
454,26081,First Digital USD,FDUSD,6.191258e+09
198,1839,BNB,BNB,4.142754e+09
256,38274,Creditlink,CDL,1.445755e+09
1027,38669,Recall,RECALL,9.705093e+08
670,7184,48 Club Token,KOGE,9.272231e+08
...,...,...,...,...
715,17445,LORDS,LORDS,0.000000e+00
605,38021,iShares Core MSCI EAFE Tokenized ETF (Ondo),IEFAON,0.000000e+00
359,19590,Defigram,DFG,0.000000e+00
1424,32536,Free Palestine,YAFA,0.000000e+00


## Итоговый скрипт

In [34]:
# импорт библиотек
import requests
import pandas as pd


def prepare_data(data, col_name, save_col=[], new_name=None):
    data = data[[col_name] + save_col].copy()
    data[col_name] = data[col_name].str.upper()
    if new_name:
        data.rename({col_name: new_name}, axis=1, inplace=True)
    return data

if __name__ == "__main__":
    # выгрузка данных по первому endpoint
    endpoint_1 = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap"
    result_1 = requests.get(endpoint_1).json()
    # определение нужных колонок
    cc_colums = ["name", "volume24h", "marketCap", "cc_id", "cc_name", "cc_symbol", "cc_dateAdded"]
    # извлечение вложенных полей из quotes
    cclist_data = pd.json_normalize(result_1['data']['cryptoCurrencyList'], 'quotes',
                                    ["id", "name", "symbol", "dateAdded"], 'cc_')[cc_colums]
    # для определения volume24h берем данные по курсу в USD
    cclist_data = cclist_data[(cclist_data['name'] == 'USD')].reset_index(drop=True)
    """
    Cоединять источники есть смысл по полю symbol, так как так получается максимальная мощность пересечения.
    В первую очередь как будто очевидно соединить по полю id, но некоторые монеты в cc и ss имеют разные id,
    например, ABBC: cc_id = 3437, ss_id = 1094).
    По полю symbol в данных coinmarketcap есть дубли, для их устранения беру токены с максимальным временем добавления
    dateAdded и максимальной капитализацией marketCap. Если и в таком случае остаются дубли, беру первый случайный токен.
    """
    cclist_data = cclist_data.sort_values(["cc_symbol", "cc_dateAdded", "marketCap"],
                                          ascending=False).groupby("cc_symbol", as_index=False).first()
    cclist_data = cclist_data.drop_duplicates(subset='cc_symbol')
    assert cclist_data[cclist_data.duplicated(subset='cc_symbol', keep=False)].shape[0] == 0
    print(f"coinmarketcap data loaded, shape = {cclist_data.shape[0]}")

    # выгрузка данных по второму endpoint
    endpoint_2 = "https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false"
    result_2 = requests.get(endpoint_2).json()
    ss_data = pd.DataFrame(result_2)
    assert ss_data[ss_data.duplicated(subset='symbol', keep=False)].shape[0] == 0
    print(f"simpleswap.io data loaded, shape = {ss_data.shape[0]}")

    # подготовка данных к соединению, выделение уникальных токенов и монет из источников
    cc_tokens = prepare_data(cclist_data, "cc_symbol", ['cc_id', 'cc_name', "volume24h"])
    ss_tokens = prepare_data(ss_data, "symbol", ["id"], 'ss_symbol')

    # склейка данных
    total_tokens = cc_tokens.merge(ss_tokens, left_on='cc_symbol', right_on='ss_symbol', how="left")

    # монеты из эндпоинта coinmarketcap, которых нет в списке доступных у обменника https://simpleswap.io.
    missed_tokens = total_tokens[total_tokens['ss_symbol'].isna()][['cc_id', 'cc_name',
                                                                    'cc_symbol', 'volume24h']]
    # сортировка по убыванию volume24h
    missed_tokens.sort_values('volume24h', ascending=False, inplace=True)
    missed_tokens.reset_index(drop=True, inplace=True)
    print(f"missed tokens processed, shape = {missed_tokens.shape[0]}")

    # выгрузка в базу
    missed_tokens.to_csv("missed_tokens.csv", index=False)
    print(f"data 'missed_tokens.csv' saved to db")

coinmarketcap data loaded, shape = 1471
simpleswap.io data loaded, shape = 2774
missed tokens processed, shape = 524
data 'missed_tokens.csv' saved to db
